In [2]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time
import pydde as d

In [94]:
#Parameters
samplenum = 1
epochs = 200
input_size = 3
output_size = 3
learning_rate = 0.01
time_length = 3; #seconds

In [95]:
# Generate simulation
dyn = d.PyDyn('test2.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.py_f(state_init)
df = dyn.py_df_dp(state_init)
dy = dyn.py_dy_dp(state_init)
print(state_init.y)
print(dy.shape)
#Sample targets only variables in z direction
y_target = np.zeros((samplenum, 3))
y_target[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
y_target[:,1] = 2
print(y_target)
p = dyn.get_p(y_target.transpose(), dyn.p_init)
y_target= torch.tensor(y_target, requires_grad= True)
print(p.dtype)

[2.15365312e-05 1.99969848e+00 0.00000000e+00 2.15757288e-05
 1.99969848e+00 0.00000000e+00 2.15562034e-05 1.99969848e+00
 0.00000000e+00]
(9, 9)
[[0.         2.         0.82907117]]
float64


## Building the custon Simulation Activation Function

In [140]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        state = dyn.compute(p)
        y_pred = torch.tensor(state.y[-3:], requires_grad = True)
        #print(f'y_pred: {y_pred.dtype}')
        
        ctx.save_for_backward(input)
        
        return y_pred, input
    
    @staticmethod
    def backward(ctx, grad_output, input):
        #print(grad_output.shape)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        state= dyn.compute(p)
        dy_dp = dyn.py_dy_dp(state)
        dy_dp = dy_dp[-3:, :]
        print(f'shape of dy/dp: {dy_dp.shape}')
        print(f'shape of grad_output: {grad_output.shape}')
        grad_output = grad_output.unsqueeze(0).t()
        print(f'shape of grad_output unsqueezed: {grad_output.shape}')
        
        grad_input = torch.tensor(dy_dp, requires_grad = True).t().mm(grad_output).t()
        #grad_input = torch.tensor(dy_dp, requires_grad = True)
        print(f'shape of grad_input: {grad_input.dtype}')

        return grad_input, None

Simulate = Simulate.apply
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, 3*time_length).double()
        self.Relu = nn.ReLU(inplace=True).double()
        self.P = nn.Linear(3*time_length, 3*time_length).double()
        #self.L_out = nn.Linear(3, 3)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.P(x)
        x = self.Relu(x)
        x, p = Simulate(x)
        #x = self.L_out(x)
        return x, p
    
model = ActiveLearn(input_size, output_size)



## FE Test

In [138]:
# Error for whole simulation
from numpy import linalg as LA

FE = 1e-6
p = torch.tensor(p, requires_grad = True).double()
y, p0 = Simulate(p)
#test = sum(y)
#test.backward()
y.backward(torch.FloatTensor([1.0, 1.0, 1.0]).double())
dy_dp = p.grad.double()
print(f'dy_dp shape = {p.grad.shape}')
print(f'dy_dp = {dy_dp}')
dy_dp_FD = np.zeros((1,len(dyn.p_init)))
#dy_dp_FD = np.zeros((3,len(dyn.p_init)))

for i in range(len(dyn.p_init)):
    dp= np.zeros(len(dyn.p_init))
    dp[i] = FE
    dp = torch.tensor(dp)
    y_p, pp = Simulate(p + dp)
    y_m, pm = Simulate(p - dp)
    y_p = y_p.detach().numpy()
    y_m = y_m.detach().numpy()
    dy_dp_FD[0, i] = sum((y_p - y_m) / (2* FE))
    #dy_dp_FD[:, i] = (y_p - y_m) / (2* FE)


print(f' dy_dp_FD = {dy_dp_FD}')
print(f' dy_dp_FD = {dy_dp_FD.shape}')
dy_dp = dy_dp.detach().numpy()
err = LA.norm(dy_dp_FD - dy_dp)
print(err)

shape of dy/dp: (3, 9)
shape of grad_output: torch.Size([3])
shape of grad_output unsqueezed: torch.Size([3, 1])
shape of grad_input: torch.float64
dy_dp shape = torch.Size([9])
dy_dp = tensor([-0.0099, -0.1359, -0.0097,  0.0200,  0.1559,  0.0250,  0.0236,  0.9489,
         0.0775], dtype=torch.float64)
 dy_dp_FD = [[-0.00987407 -0.1358899  -0.00967889  0.02002359  0.15594092  0.02497684
   0.02363982  0.94892009  0.07746342]]
 dy_dp_FD = (1, 9)
1.1611717294272575e-09


In [9]:
from numpy import linalg as LA

#Calculate dy_dp with FE
FE = 1e-6
dy_dp = sim.dy_dp(sim.y, sim.ydot, sim.yddot, sim.p)
dy_dp_FD = np.zeros((len(sim.p),len(sim.p)))

for i in range(len(sim.p)):
    dp= np.zeros(len(sim.p))
    dp[i] = FE
    pos = sim.p + dp
    y_p = sim.compute(pos)
    y_m = sim.compute(sim.p - dp)
    dy_dp_FD[:, i] = (y_p - y_m) / (2* FE)
err = LA.norm(dy_dp_FD - dy_dp)
print(err)


0.0002908451455664775


In [31]:
x = torch.tensor(1.0, requires_grad = True)
y = torch.tensor(2.0)
z = x * y

# Displaying
for i, name in zip([x, y, z], "xyz"):
    print(f"{name}\ndata: {i.data}\nrequires_grad: {i.requires_grad}\n\
grad: {i.grad}\ngrad_fn: {i.grad_fn}\nis_leaf: {i.is_leaf}\n")

x
data: 1.0
requires_grad: True
grad: None
grad_fn: None
is_leaf: True

y
data: 2.0
requires_grad: False
grad: None
grad_fn: None
is_leaf: True

z
data: 2.0
requires_grad: True
grad: None
grad_fn: <MulBackward0 object at 0x1245503c8>
is_leaf: False



In [4]:
import os
print(os.__file__)
print(d.__file__)
print(np.__file__)
print(nn.__file__)
print(pd.__file__)

/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.7/lib/python3.7/os.py
/Users/nicovonau/Code/thesis_pytorch/pydde.cpython-37m-darwin.so
/Users/nicovonau/Library/Python/3.7/lib/python/site-packages/numpy/__init__.py
/Users/nicovonau/Library/Python/3.7/lib/python/site-packages/torch/nn/__init__.py
/Users/nicovonau/Library/Python/3.7/lib/python/site-packages/pandas/__init__.py
